In [20]:
import os
import re
import csv
import json
import urllib.request as request

We have worked on Google Colab, so need to mount drive. Remove this cell if you run on local computer.

In [17]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Change the root to your project directory to run this notebook.

In [18]:
root = '/content/drive/My Drive/DS_World Happiness'
os.chdir(os.path.join(root, 'code'))

In [19]:
worldbank_indicators = [
    'EN.POP.DNST',  # Population density
    'NY.GDP.PCAP.CD',  # GDP per capita
    'EN.ATM.PM25.MC.M3',  # PM2.5 air pollution, mean annual exposure (micrograms per m3)
    'AG.LND.FRST.ZS',  # Forest area (% of land area)
    'FP.CPI.TOTL',  # Consumer price index
]

undp_indicators = [
    103006,  # Mean years of schooling
    69206,  # Life expectancy at birth (years)
    137906,  # Gender development index (GDI)
    195606,  # CO2 emissions per capita
    140606,  # Unemployment rate
]

who_indicators = [
    'WHOSIS_000002',  # Healthy life expectancy
    'GHED_CHEGDP_SHA2011',  # Current health expenditure (% GDP)
    'MH_12',  # Age-standardized suicide rates (per 100 000 population)
]

Here, we wrote a mini verion of OpenRefine for our project. How it works is written in the report

In [21]:
# A mini open-refine
def split_words(text):
    return [i for i in re.split(r'[^\w]+', text) if i != '']

def match_countries(names, altnames, filename='matching_result.json'):
    mapping = {}

    # First, match countries with exact name and altname
    for i, name in enumerate(names):
        if name in altnames:
            mapping[name] = name
            altnames.remove(name)
            names[i] = None
    names = [i for i in names if i is not None]

    # Then, match countries whose names is a part of altname
    for i, name in enumerate(names):
        candidates = []
        for altname in altnames:
            if name.lower() in altname.lower():
                candidates.append(altname)
        if candidates:
            # If multiple altnames are available, use the one with
            # the most similar length to name
            altname = min(candidates, key=lambda x: abs(len(x) - len(name)))
            mapping[name] = altname
            altnames.remove(altname)
            names[i] = None
    names = [i for i in names if i is not None]

    # Next, match the countries with matching words
    for i, name in enumerate(names):
        candidates = {}
        for word in split_words(name.lower()):
            for altname in altnames:
                if word in altname.lower():
                    if altname in candidates:
                        candidates[altname] += 1
                    else:
                        candidates[altname] = 1
        for altname in altnames:
            for word in split_words(altname.lower()):
                if word in name.lower():
                    if altname in candidates:
                        candidates[altname] += 1
                    else:
                        candidates[altname] = 1

        if candidates:
            # Priotize names with most matches
            altname = min(candidates, key=lambda x: (-candidates[x], abs(len(x) - len(name))))
            mapping[name] = altname
            altnames.remove(altname)
            names[i] = None
    names = [i for i in names if i is not None]

    # Finally, match the countries with most similarity in characters
    for i, name in enumerate(names):
        if not altnames:
            break
        candidates = {}
        for altname in altnames:
            if len(name) < len(altname):
                a, b = name.lower(), altname.lower()
            else:
                a, b = altname.lower(), name.lower()

            count = 0
            for char in a:
                if char in b:
                    count += 1
            candidates[altname] = count / len(a)
        altname = min(candidates, key=lambda x: (-candidates[x], abs(len(x) - len(name))))
        mapping[name] = altname
        altnames.remove(altname)
        names[i] = None
    names = [i for i in names if i is not None]

    with open(filename, 'w') as file:
        json.dump({
            'mapping': mapping,
            'unmatched names': names,
            'unmatched altnames': altnames,
        }, file, indent=4)

    input(f'Please modify and confirm the mapping in {filename} before continuing, remove names without a corresponding altname')
    with open(filename) as file:
        mapping = json.load(file)['mapping']
    return mapping

In [22]:
# The format of dataset is indicators -> country names -> data
dataset = {}
countries = {}

We load all countries with associated ISO3 code by using World Bank data's API

In [23]:
# Load all countries
with request.urlopen('https://api.worldbank.org/v2/country?format=json&per_page=1000000') as web:
    metadata, countries_list = json.load(web)
    for country in countries_list:
        # Because the list from the API also include regions
        # we try to exclude as many regions as possible
        if country['longitude'] != '' or country['region']['id'] != 'NA':
            countries[country['name']] = country['id']

Here we load the happiness index and regional indicator data from the csv downloaded from [the world happiness report website](https://worldhappiness.report/ed/2020/).

Because the country names there may be different from the ones in World Bank data's API and we don't have the ISO3 codes corresponding to those names, out mini OpenRefine is used. The program will ask the user to modify a file containing the mapping from names in the world happiness report to names in World Bank data's API.

In [24]:
happinessdata = {}
with open('../data/happiness.csv', newline='') as file:
    reader = csv.reader(file)
    next(reader)  # Ignore first row
    for row in reader:
        happinessdata[row[0]] = [row[1], row[2]]

# Mapping from names in happiness data to names in worldbank data
mapping = match_countries(list(happinessdata), list(countries), filename='happinessmapping.json')
# A mapping from iso3 code to names in happiness report
countries_list = {}
for happinessdataname, worldbankdataname in mapping.items():
    countries_list[countries[worldbankdataname]] = happinessdataname

# Add happiness and region data
dataset['Region'] = {}
dataset['Happiness Index'] = {}
for country, data in happinessdata.items():
    dataset['Region'][country] = data[0]
    dataset['Happiness Index'][country] = data[1]

Please modify and confirm the mapping in happinessmapping.json before continuing, remove names without a corresponding altname


Next, we download data from World Bank data, UNDP data and WHO using their APIs

In [25]:
# Get data from worldbank
for indicator in worldbank_indicators:

    # Worldbank uses int16 for per_page data, so 32767 is max
    with request.urlopen(f'https://api.worldbank.org/v2/country/all/indicator/{indicator}?'
                         f'format=json&per_page=32767&date=2017:2019') as web:
        metadata, multi_indicators = json.load(web)
        indicator_data = {}
        dataset[multi_indicators[0]['indicator']['value']] = indicator_data

        for single_indicator in multi_indicators:
            country_code = single_indicator['countryiso3code']
            country = countries_list.get(country_code)
            if country is None:
                continue
            if country not in indicator_data:
                indicator_data[country] = [None] * 3

            year = int(single_indicator['date'])
            value = single_indicator['value']
            indicator_data[country][year - 2017] = value

In [26]:
# Get data from undp
undp_indicators_query = ','.join(map(str, undp_indicators))

with request.urlopen(f'http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/'
                     f'indicator_id={undp_indicators_query}/year=2017,2018,2019/structure=icy') as web:
    fulldata = json.load(web)
    indicators = fulldata['indicator_name']
    for id, data in fulldata['indicator_value'].items():
        indicator_data = {}
        dataset[indicators[id]] = indicator_data
        for country_code, country_data in data.items():
            if country_code in countries_list:
                indicator_data[countries_list[country_code]] = [country_data.get(str(i)) for i in range(2017, 2020)]

In [27]:
# Get data from WHO
filter_string = (
    "SpatialDimType eq 'COUNTRY'"  # Get country-specific data
    " and TimeDimType eq 'YEAR'"  # Get yearly data
    " and (TimeDim eq 2017 or TimeDim eq 2018 or TimeDim eq 2019)"  # Only for the year 2017, 2018 and 2019
    " and (Dim1Type eq null or (Dim1Type eq 'SEX' and Dim1 eq 'BTSX'))"  # Get data from both sexes
).replace("'", '%27').replace(' ', '%20')

for indicator in who_indicators:
    indicator_data = {}
    with request.urlopen(f'https://ghoapi.azureedge.net/api/Indicator?'
                         f'$filter=IndicatorCode%20eq%20%27{indicator}%27') as web:
        dataset[json.load(web)['value'][0]['IndicatorName']] = indicator_data

    with request.urlopen(f"https://ghoapi.azureedge.net/api/{indicator}?"
                         f"$filter={filter_string}") as web:
        data = json.load(web)['value']
        for country in data:
            country_code = country['SpatialDim']
            country_name = countries_list.get(country_code)
            if country_name is None:
                continue
            if country_name not in indicator_data:
                indicator_data[country_name] = [None] * 3
            
            year = country['TimeDim']
            value = country['NumericValue']
            indicator_data[country_name][year - 2017] = value

Then we load the downloaded csv file from [United Nation Office on Drugs and Crime](https://dataunodc.un.org/content/prison-population-national-vs-regional-estimates). We used our mini version of OpenRefine here again, this time for mapping country names in the crime data file to country names in happiness report.

In [28]:
# Add crime data
crimedata = {}
with open('../data/crime.csv', newline='') as file:
    reader = csv.reader(file)
    next(reader)  # Ignore first row
    # row[1] is country name
    # row[3] is level, must be "Country"
    # row[9] is year, chose 2017, 2018 and 2019 only
    # row[12] is value
    for row in reader:
        year = int(row[9])
        if row[3] == 'Country' and 2017 <= year <= 2019:
            name = row[1]
            if name not in crimedata:
                crimedata[name] = [None] * 3
            crimedata[name][year - 2017] = float(row[12])

# Mapping from names in crime data to names in happiness data
mapping = match_countries(list(crimedata), list(happinessdata), filename='crimemapping.json')

dataset['Person held per 100,000 population'] = {}
for crimedataname, happinessdataname in mapping.items():
    dataset['Person held per 100,000 population'][happinessdataname] = crimedata[crimedataname]

Please modify and confirm the mapping in crimemapping.json before continuing, remove names without a corresponding altname


After collected all required data, we calculate its mean

In [29]:
# Calculate mean value
for indicator, data in dataset.items():
    if indicator != 'Region' and indicator != 'Happiness Index':
        for country, values in data.items():
            if values[0] is None and values[1] is None and values[2] is None:
                data[country] = None
            else:
                values = [i for i in values if i is not None]
                data[country] = sum(values) / len(values)


And finally rearrange the columns and write to a csv file.

In [30]:
header = ['', 'Country', 'Regional Indicator']
lines = [[i, data[0], data[1]] for i, data in enumerate(dataset['Region'].items())]
dataset.pop('Region')

for indicator, data in dataset.items():
    # Happiness index will be the last indicator
    if indicator != 'Happiness Index':
        header.append(indicator)
        for line in lines:
            name = line[1]
            value = data.get(name)
            line.append(value if value is not None else '')
header.append('Happiness Index')
for line in lines:
    line.append(dataset['Happiness Index'][line[1]])


with open('../data/data_processed.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(lines)